# Curate PDB assemblies from CIF 
2023-2-11

This notebook curates datasets of assemblies that may contain multiple protein chains and multiple ligand chains. These are intended to be used in phase 3 of training of the fold & dock 3 model.

The code below does:

1. Data curation, consisting of the following steps:

 - Ivan's CIFUtils parser is used to process all PDB entries downloaded in mmCIF format in Dec 2022. (During training, only examples from before 2020-04-30 are used)
 - Make a list of all small-molecule ligands, defined as sets of tuples (chain ID, residue number) with chain type "nonpoly" and containing covalent bonds to each other. Each of these "query" ligands will be the basis of a training example. Also identify if the ligand has covalent bonds to a protein chain.
 - Remove a manually chosen set of non-biological ligands (based on residue name)
 - Remove PDB ID / ligand name combinations that are not listed in PDBBind/BioLiP, to avoid low quality training examples. This removes 60-75% of ligands, so may be worth revisiting to see if some of the excluded examples are worth including in training.
 - Associate each query ligand with nearby protein and ligand chains. The protein chain with the most contacts is the "main" chain for this training example and its sequence is used for various filtering and merging operations later.
 - Exclude examples with protein chain similarity to Astex and CASF2016 examples, so we can use these sets for evaluation. We don't take ligand similarity into account.
 - Various filtering steps to avoid errors during featurization & training:
   - Remove covalent protein-ligand bonds that seem non-biological (O-O, F-F, -H2O, etc)
   - Remove examples where ligands are <1A from a protein atom
   - Remove examples where protein chains have a different number of residues from their pre-computed MSAs (these are mistakes during MSA curation).
   - Remove nucleic acid chains (these can be added back later, avoiding for now to simplify data loaders)
   - Label ligands with # of atoms. These are used to filter ligands larger than the crop size to control GPU memory usage
   
2. Dataset splitting. We make different mutually exclusive datasets with chemically/technically different ligands to control the proportions seen during training:

 - Protein / organic ligands (sm_compl)
 - Protein / metal ions (metal_compl)
 - Protein / multi-residue ligands (sm_compl_multi)
 - Protein / covalent ligands (sm_compl_covale)
 - Protein / ligand assemblies (more than 1 protein or ligand chain) (sm_compl_asmb)

3. Make a strict validation set for the protein / organic ligands where ligands with tanimoto similarity > 0.85 to the training set are removed.

## Imports

In [1]:
import sys, os, glob, pickle, gzip
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
script_dir = os.getcwd()

In [3]:
sys.path.insert(0,script_dir+'/../')
sys.path.insert(0,script_dir+'/../rf2aa/')
import analysis_util
from parsers import *
from util import is_atom

In [4]:
from curation_utils import *

## Pre-parse Ivan's dataset

In [3]:
sys.path.insert(0,'/home/jue/git/chemnet/arch.22-10-28/')
sys.path.insert(0,'/home/jue/git/chemnet/arch.22-10-28/pdb/')
import cifutils

/software/conda/envs/dlchem/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
Parser = cifutils.CIFParser()

In [5]:
basedir = '/databases/rcsb/cif/'

In [6]:
subdirs =  glob.glob(basedir+'/*/')

In [7]:
len(subdirs)

1060

In [8]:
filenames = []

In [9]:
for subdir in subdirs:
    filenames += glob.glob(subdir+'/*.cif.gz')    

In [10]:
len(filenames)

202197

In [11]:
with open('all_rcsb_cif.txt','w') as outf:
    for fn in filenames:
        print(fn, file=outf)

In [12]:
filenames = [line.strip() for line in open('all_rcsb_cif.txt').readlines()]

In [13]:
istart = 0
num = 1

In [15]:
for fn in filenames[istart:istart+num]:
    try:
        chains,asmb,covale,modres = Parser.parse(fn)
        outfn = fn.replace('/databases/rcsb/cif/','/projects/ml/RF2_allatom/rcsb/pkl_v2/')
        outfn = outfn.replace('.cif.gz','.pkl.gz')
        outdir = os.path.dirname(outfn)
        os.makedirs(outdir, exist_ok=True)
        pickle.dump((chains,asmb,covale,modres),gzip.open(outfn,'wb'))
    except Exception as e:
        print(fn)
        print(repr(e))

Put the above code into a python script so it can be run in parallel on many nodes to process the full dataset

Generate a list of commands to run

In [14]:
num = 500

In [15]:
with open('cmds_parse_cifs.txt','w') as outf:
    for istart in range(0, len(filenames), num):
        print(f'source activate dlchem; python parse_cifs.py -istart {istart} -num {num}',
              file=outf)

In [30]:
filenames = glob.glob('/projects/ml/RF2_allatom/rcsb/pkl_v2/*/*.pkl.gz')

In [31]:
len(filenames)

202183

## Get ligands
Use Ivan's parsed cif data structures to identify ligands ("nonpoly" residues) and save a list with information about them.

In [5]:
import time, pickle, gzip

In [25]:
cif_files = glob.glob('/projects/ml/RF2_allatom/rcsb/pkl/*/*.pkl.gz')

In [26]:
len(cif_files)

202183

In [27]:
from collections import Counter

In [28]:
from importlib import reload
import curation_utils
reload(curation_utils)
from curation_utils import *

In [37]:
records = []
ct = 0
start_time = time.time()

for fn in cif_files:
    try:                                                                                              
        pdbid = os.path.basename(fn).replace('.pkl.gz','')
        chains, asmb, covale, modres = pickle.load(gzip.open(fn))

        ligands, lig_covale = get_ligands(chains, covale)

        # add ligands in order of assembly
        # this will make loading & filtering later more efficient
        for i_a in asmb:
            for lig, lcovale in zip(ligands,lig_covale):
                ligand_chids = set([res[0] for res in lig])

                if not ligand_chids.issubset(set([x[0] for x in asmb[i_a]])):
                    continue

                records.append(dict(
                    PDBID=pdbid,
                    LIGAND=lig,
                    ASSEMBLY=i_a,
                    COVALENT=lcovale,
                ))
    except Exception as e:
        print(e)
    
    ct += 1
    if ct % 100 == 0:
        print(ct, time.time()-start_time)
    
    if any([len(lcovale)>0 for lcovale in lig_covale]): break

In [32]:
df = pd.DataFrame.from_records(records)

Put the above code into a python script so it can be run in parallel on many nodes to process the full dataset

Generate a list of commands to run

In [10]:
num = 500

In [19]:
with open('cmds_get_ligands.txt','w') as outf:
    for istart in range(0, len(cif_files), num):
        fn = f'results_get_ligands/ligands{istart}.csv'
        if not os.path.exists(fn):
            print(f'source activate dlchem; python get_ligands.py -istart {istart} -num {num}',
                  file=outf)

results_get_ligands/ligands21500.csv
results_get_ligands/ligands22000.csv
results_get_ligands/ligands22500.csv
results_get_ligands/ligands46000.csv
results_get_ligands/ligands49000.csv
results_get_ligands/ligands57000.csv
results_get_ligands/ligands62500.csv
results_get_ligands/ligands70000.csv
results_get_ligands/ligands111000.csv
results_get_ligands/ligands115000.csv
results_get_ligands/ligands131000.csv
results_get_ligands/ligands140500.csv
results_get_ligands/ligands168500.csv
results_get_ligands/ligands193500.csv


Compile the final list

In [9]:
len(cif_files)

201835

In [20]:
for i in range(0,len(cif_files),num):
    fn = f'results_get_ligands/ligands{i}.csv'
    if not os.path.exists(fn):
        print(fn)

In [21]:
df = pd.concat([pd.read_csv(f'results_get_ligands/ligands{i}.csv',index_col=0) 
                for i in range(0,len(cif_files),num)])

In [22]:
df.shape

(1961071, 4)

In [23]:
df.to_csv('ligands.csv',index=None)

## Filter non-bio ligands

Remove these "non-biological" ligands. Previously curated by Gyu Rie and Linna, from `/home/gyurie/mpnn_ligand/bad_lig_list`. They were trying to exclude metal ions so I add these back, based on this list from the BioLiP website (extracted from the url when you click on their link for "PDB entries with metals")

Rohith and I added PO4, because these examples are numerous and poorly predicted

In [24]:
biolip_list = ['NUC','ZN','CA','MG','III','MN','FE','CU','SF4','FE2','CO','FES','GOL','NA',  
            'CL','K','CU1','GOL','XE','NO2','EDO','NI','BR','CD','O','CS','NO','TL','HG','UNL','KR',
            'SR','RB','F','AG','AR','U','AU','MO','SE','GD','YB','VX','SM','LI','RE','N','W','OS','HO','PI']
LA_list = ['EDO','PG4','OGA','SO4','HEZ','FEO','CL','DMS','ACT','MPD','GOL' ,'NH2','CUA','SIW','PGW','IOD','BR','3NI','ZRW','78M','UNX','nan']
rohith = ['MES','CCN','PO4']
metals = ['LA','NI','3CO','K','CR','ZN','CD','PD','TB','YT3','OS','EU','NA','RB','W','YB','HO3',
          'CE','MN','TL','LI','MN3','AU3','AU','EU3','AL','3NI','FE2','PT','FE','CA','AG','CU1',
          'LU','HG','CO','SR','MG','PB','CS','GA','BA','SM','SB','CU','MO','CU2']

In [25]:
exclude = set(biolip_list+LA_list+rohith)-set(metals)

In [35]:
df = pd.read_csv('ligands.csv')

In [30]:
df['LIGAND'] = df['LIGAND'].apply(lambda x: eval(x))

In [31]:
df.shape

(1961071, 4)

In [32]:
df_filt = df[~df['LIGAND'].apply(lambda x: x[0][2] in exclude)]

In [33]:
df_filt.shape

(1497746, 4)

In [34]:
df_filt.to_csv('ligands_filt.csv',index=None)

## Download PDBBind / BioLiP
2022-12-13

These are curated lists of biologically relevant ligands

### PDBBind

In [54]:
import re
from collections import OrderedDict

In [55]:
lig_re = re.compile('\((.*?)\)')

In [56]:
records = []
with open('/home/jue/dev/rfaa/index/INDEX_general_PL.2020') as f:
    lines = f.readlines()
    for line in lines:
        if not line.startswith('#'):
            tokens = line.split('//')
            desc = tokens[1]
            ligand = lig_re.findall(desc)[0]
            tokens = tokens[0].split()
            records.append(OrderedDict(
                pdb=tokens[0],
                resolution=tokens[1],
                year=tokens[2],
                affinity=tokens[3],
                ligand=ligand,
                description=desc
            ))

In [57]:
pdbbind = pd.DataFrame.from_records(records)

In [58]:
pdbbind.shape

(19443, 6)

In [112]:
pdbbind.to_csv('pdbbind_20221213.csv')

### BioLiP

In [80]:
import requests

In [96]:
for datestr in pd.date_range(start='2013-3-13', end='2022-03-30', freq='7D'):
    url = f'https://zhanggroup.org/BioLiP/weekly/BioLiP_{str(datestr).split()[0]}.txt'
    r = requests.get(url, allow_redirects=True)
    fn = os.path.basename(url)
    open(fn, 'wb').write(r.content)
    print(fn)

BioLiP_2013-03-13.txt
BioLiP_2013-03-20.txt
BioLiP_2013-03-27.txt
BioLiP_2013-04-03.txt
BioLiP_2013-04-10.txt
BioLiP_2013-04-17.txt
BioLiP_2013-04-24.txt
BioLiP_2013-05-01.txt
BioLiP_2013-05-08.txt
BioLiP_2013-05-15.txt
BioLiP_2013-05-22.txt
BioLiP_2013-05-29.txt
BioLiP_2013-06-05.txt
BioLiP_2013-06-12.txt
BioLiP_2013-06-19.txt
BioLiP_2013-06-26.txt
BioLiP_2013-07-03.txt
BioLiP_2013-07-10.txt
BioLiP_2013-07-17.txt
BioLiP_2013-07-24.txt
BioLiP_2013-07-31.txt
BioLiP_2013-08-07.txt
BioLiP_2013-08-14.txt
BioLiP_2013-08-21.txt
BioLiP_2013-08-28.txt
BioLiP_2013-09-04.txt
BioLiP_2013-09-11.txt
BioLiP_2013-09-18.txt
BioLiP_2013-09-25.txt
BioLiP_2013-10-02.txt
BioLiP_2013-10-09.txt
BioLiP_2013-10-16.txt
BioLiP_2013-10-23.txt
BioLiP_2013-10-30.txt
BioLiP_2013-11-06.txt
BioLiP_2013-11-13.txt
BioLiP_2013-11-20.txt
BioLiP_2013-11-27.txt
BioLiP_2013-12-04.txt
BioLiP_2013-12-11.txt
BioLiP_2013-12-18.txt
BioLiP_2013-12-25.txt
BioLiP_2014-01-01.txt
BioLiP_2014-01-08.txt
BioLiP_2014-01-15.txt
BioLiP_201

In [107]:
df_s = []
for datestr in pd.date_range(start='2013-3-06', end='2022-03-30', freq='7D'):
    fn = f'biolip/BioLiP_{str(datestr).split()[0]}.txt'
    
    dat = open(fn).read()
    if len(dat)==0 or '404 Not Found' in dat: continue
    
    tmp = pd.read_csv(fn,delimiter='\t',header=None)
    tmp.columns = ['pdb_id','pdb_chain','resolution','binding_site_id','ligand_id','ligand_chain','ligand_serial_number',
                  'bs_res','bs_res_renum','cat_site_res','cat_site_res','EC','GO','binding_affinity','binding_affinity_MOAD',
                  'binding_affinity_pdbbind','binding_affinity_bindingDB','uniprot','pubmed','receptor_seq']
    df_s.append(tmp)
biolip = pd.concat(df_s)

In [108]:
biolip.shape

(477226, 20)

In [111]:
biolip.to_csv('biolip_combined_20221213.csv')

## Filter to PDBBind / BioLiP

In [48]:
pdbbind = pd.read_csv('pdbbind_20221213.csv', na_filter=None)
biolip = pd.read_csv('biolip_combined_20221213.csv', na_filter=None)

In [49]:
biolip_entries = biolip[['pdb_id','ligand_id']].rename(columns={'pdb_id':'PDBID','ligand_id':'LIGNAME'})
biolip_entries['KEEP'] = True

In [50]:
pdbbind_entries = pdbbind[['pdb','ligand']].rename(columns={'pdb':'PDBID', 'ligand':'LIGNAME'}).drop_duplicates()
pdbbind_entries['KEEP'] = True

In [51]:
to_keep = biolip_entries.append(pdbbind_entries)
to_keep = to_keep.drop_duplicates()

/scratch/jue/28700558/ipykernel_1480857/307790726.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  to_keep = biolip_entries.append(pdbbind_entries)


In [52]:
to_keep.shape

(166142, 3)

In [75]:
df = pd.read_csv('ligands_filt.csv')

In [ ]:
df['LIGAND'] = df['LIGAND'].apply(lambda x: eval(x))

In [54]:
df.shape

(1497746, 4)

In [56]:
records = []
for i,row in df.iterrows():
    for lig in row['LIGAND']:
        newrow = row.copy()
        newrow['LIGNAME'] = lig[2]
        records.append(newrow)

In [57]:
df2 = pd.DataFrame.from_records(records)

In [58]:
df2.shape

(1595933, 5)

In [59]:
tmp = df2.merge(to_keep, on=['PDBID','LIGNAME'], how='left')

In [60]:
tmp.shape

(1595933, 6)

In [61]:
df3 = tmp[tmp['KEEP']==True]

In [62]:
df3.shape

(422178, 6)

In [63]:
df3 = df3.drop(['LIGNAME','KEEP'], axis=1)

In [64]:
df3['LIGAND'] = df3['LIGAND'].apply(lambda x: str(x))

In [65]:
df3 = df3.drop_duplicates()

In [66]:
df3.shape

(408890, 4)

In [75]:
df3 = df3.sort_values(['PDBID','ASSEMBLY'])

In [67]:
df3.to_csv('ligands_filt.csv',index=None)

## Get binding partners
Identify which protein and ligand chains are closest to each query ligand.

Also do a few steps which were previously done in later stages (it's more efficient to combine these):
 - count ligand atoms (resolved & unresolved)
 - filter covalent ligands on some quality criteria

In [9]:
import pickle, gzip, time
from util import cif_ligand_to_xyz, get_ligand_atoms_bonds

In [10]:
def is_weird(covalents):
    """Detects non-biological bonds"""
    has_oxygen_oxygen_bond = any([a1[3][0]=='O' and a2[3][0]=='O' for (a1,a2) in covalents])
    has_fluorine_fluorine_bond = any([a1[3][0]=='F' and a2[3][0]=='F' for (a1,a2) in covalents])
    is_oxy_hydroxy = any([a1[2]=='O' or a2[2]=='O' or \
                          a1[2]=='OH' or a2[2]=='OH' or \
                          a1[2]=='HOH' or a2[2]=='HOH' for (a1,a2) in covalents])
    return has_oxygen_oxygen_bond or has_fluorine_fluorine_bond or is_oxy_hydroxy

def is_clashing(qlig_xyz_valid, xyz_chains, mask_chains, asmb_xforms):
    """Detects if a ligand is within 1A of any protein in its assembly."""
    for xyz, mask in zip(xyz_chains, mask_chains):
        for i_xf, (xf_ch, xf) in enumerate(asmb_xforms):
            if xf_ch != ch.id: continue
            xf = torch.tensor(xf).float()
            u,r = xf[:3,:3], xf[:3,3]
            xyz_xf = torch.einsum('ij,raj->rai', u, xyz) + r[None,None]

            atom_xyz = xyz_xf[:,:NHEAVY][mask[:,:NHEAVY].numpy(),:]
            dist = torch.cdist(qlig_xyz_valid, atom_xyz, compute_mode='donot_use_mm_for_euclid_dist')
            if (dist<1).any():
                return True
    return False

def filter_partners(partners, chains, xyz_chains, mask_chains, asmb_xforms, covale, row):
    new_partners = []
    for p in partners:
        if p[-1]=='nonpoly':
            # remove covalent partners with non-biological bonds
            bonds = []
            for bond in covale:
                if any([bond.a[:3]==res or bond.b[:3]==res for res in p[0]]):
                    bonds.append((bond.a, bond.b))
            if len(bonds)>0:
                if is_weird(bonds):
                    print('non-biological protein-ligand bond in partner',row['PDBID'],p)
                    continue
            # remove partners with clash to protein
            plig = p[0]
            lig_xyz, lig_mask, lig_seq, lig_chid, lig_resi, lig_chxf = \
                get_ligand_xyz(chains, asmb_xforms, plig,
                               seed_ixf=dict(p[1])[plig[0][0]])

            lig_xyz_valid = lig_xyz[lig_mask]
            clash = is_clashing(lig_xyz_valid, xyz_chains, mask_chains, asmb_xforms)
            if clash:
                print('clash between partner ligand and protein',row['PDBID'],p)
                continue
        new_partners.append(p)
    return new_partners

### Run

In [11]:
df = pd.read_csv('ligands_filt.csv')

In [12]:
df['LIGAND'] = df['LIGAND'].apply(lambda x: eval(x))
df['COVALENT'] = df['COVALENT'].apply(lambda x: eval(x))

In [13]:
df.shape

(408890, 4)

In [17]:
records = []
start_time = time.time()

pdbid_prev = None
i_a_prev = None
for i,row in df2.iterrows():
    pdbid = row['PDBID']
    if pdbid!=pdbid_prev:
        chains,asmb,covale,modres = pickle.load(gzip.open(f'/projects/ml/RF2_allatom/rcsb/pkl/{pdbid[1:3]}/{pdbid}.pkl.gz'))
        ligands, lig_covale = get_ligands(chains, covale)

    # remove covalent examples with non-biological bonds
    if len(row['COVALENT'])>0:
        if is_weird(row['COVALENT']):
            print('non-biological protein-ligand bond',row['PDBID'],row['LIGAND'])
            continue
            
    i_a = str(row['ASSEMBLY'])
    asmb_xforms = asmb[i_a]
    asmb_xform_chids = [x[0] for x in asmb_xforms]
    asmb_chains = [chains[i_ch] for i_ch in set(asmb_xform_chids)]

    if pdbid!=pdbid_prev or i_a!=i_a_prev:
        # featurize all protein chains
        xyz_chains, mask_chains = [], []
        for ch in asmb_chains:
            if ch.type != 'polypeptide(L)': continue
            xyz, mask, seq, chid, resi, unrec_elements = chain_to_xyz(ch)
            xyz_chains.append(xyz)
            mask_chains.append(mask)
        
    pdbid_prev = pdbid
    i_a_prev = i_a

    # assembly must have at least one protein and ligand chain
    if not {'polypeptide(L)','nonpoly'}.issubset(set([ch.type for ch in asmb_chains])):
        continue

    # get query ligand coordinates (one copy of it in this assembly)
    query_ligand = row['LIGAND']
    qlig_xyz, qlig_mask, qlig_seq, qlig_chid, qlig_resi, qlig_chxf = \
        get_ligand_xyz(chains, asmb_xforms, query_ligand)
    
    # filter out query ligand residues that weren't loaded (all atoms have 0 occupancy)
    query_ligand = [res for res in query_ligand if res[0] in [x[0] for x in qlig_chxf]]
    row['COVALENT'] = [(a,b) for (a,b) in row['COVALENT'] if any([a[:3]==res or b[:3]==res for res in query_ligand])]
    
    # remove empty ligands
    if qlig_xyz.numel()==0: continue
    qlig_xyz_valid = qlig_xyz[qlig_mask[:,1],1]
    if qlig_xyz_valid.numel()==0: continue

    # remove examples with clashes between query ligand and protein chains
    clash = is_clashing(qlig_xyz_valid, xyz_chains, mask_chains, asmb_xforms)
    if clash:
        print('clash between query ligand and protein',row['PDBID'], row['LIGAND'], row['ASSEMBLY'])
        continue

    # get list of all coordinate-transformed protein chains in assembly that contact this ligand
    prot_na_contacts = get_contacting_chains(asmb_chains, asmb_xforms, 
                                     qlig_xyz_valid, qlig_chxf)

    # get list of all coordinate-transformed ligands in assembly that contact this ligand
    lig_contacts = get_contacting_ligands(ligands, chains, asmb_xforms, 
                                          query_ligand, qlig_xyz_valid, qlig_chxf)

    prot_contacts = [x for x in prot_na_contacts if x[-1]=='polypeptide(L)' and x[2]>0]
    if len(prot_contacts) == 0:
        continue # no protein <5A of ligand, don't use for training

    # pool all contacting objects, sort from most to least contacts (then lowest to highest min distance)
    partners = sorted(prot_na_contacts+lig_contacts, key=lambda x: (x[2], -x[3]), reverse=True)

    # filter partners
    new_partners = filter_partners(partners, chains, xyz_chains, mask_chains, asmb_xforms, covale, row)

    # save results
    new_row = row.copy()
    new_row['LIGAND'] = query_ligand # update in case we removed some residues with no resolved atoms
    new_row['ASSEMBLY'] = i_a
    new_row['PROT_CHAIN'] = prot_contacts[0][0] # most-contacting protein chain
    new_row['LIGXF'] = qlig_chxf
    new_row['PARTNERS'] = new_partners
    new_row['LIGATOMS'] = qlig_xyz.shape[0]
    new_row['LIGATOMS_RESOLVED'] = qlig_mask.sum().item()
    
    records.append(new_row)

    if i%50 == 0:
        print(i, time.time()-start_time)

In [254]:
df2 = pd.DataFrame.from_records(records)

Put the above in a standalone script to run in parallel on digs.

Generate a list of commands to run

In [37]:
num = 300

In [256]:
with open('cmds_get_partners.txt','w') as outf:
    for istart in range(0, len(df), num):
        if not os.path.exists(f'results_get_partners/ligands_partners{istart}.csv'):
            print(f'source activate dlchem; python get_partners.py -istart {istart} -num {num}',
                  file=outf)

In [257]:
for istart in range(0, len(df), num):
    if not os.path.exists(f'results_get_partners//ligands_partners{istart}.csv'):
        print(istart)

Compile the final list

In [258]:
filenames = [f'results_get_partners/ligands_partners{i}.csv'
             for i in range(0,len(df),num)]

In [259]:
df2 = pd.concat([pd.read_csv(fn, index_col=0, na_filter=False) for fn in filenames])

In [260]:
df2.shape

(383557, 9)

In [261]:
df2.to_csv('ligands_partners.csv',index=None)

## Merge in protein seq clusters

Cross-reference list of multi-residue examples with existing metadata to allow splitting into train & valid sets, and assign protein cluster IDs and ligand cluster sizes, to allow sampling in our current weighted training example sampling framework

In [327]:
df = pd.read_csv('ligands_partners.csv', na_filter=False)

In [265]:
df['CHAINID'] = df.apply(lambda row: str(row['PDBID'])+'_'+row['PROT_CHAIN'], axis=1)

In [272]:
df.shape

(383557, 10)

In [282]:
df2 = pd.read_csv('/projects/ml/TrRosetta/PDB-2021AUG02/list_v02.csv')

In [283]:
df2.shape

(534579, 7)

In [284]:
df2_subset = df2[['CHAINID','DEPOSITION','RESOLUTION','HASH',
                  'CLUSTER','SEQUENCE','LEN_EXIST']].drop_duplicates('CHAINID')

In [285]:
df3 = df2_subset.merge(df,on='CHAINID')

In [286]:
df3.shape

(360930, 16)

Check which entries get excluded after merging to our old dataset

In [287]:
missing = set(df['CHAINID'].values)-set(df2['CHAINID'].values)

In [288]:
len(missing)

10616

In [287]:
df4 = df[df['CHAINID'].isin(missing)]

In [288]:
df4.to_csv('missing.csv')

The missing entries are newer (2021-) PDBs as well as too-short chains (<60aa). Ivan didn't do sequence clustering on shorter sequences so we will need to leave these out for now.

In [289]:
df3.to_csv('ligands_partners_clustered.csv',index=None)

## Exclude astex / CASF2016 by protein cluster

In [292]:
df = pd.read_csv('ligands_partners_clustered.csv', na_filter=False)

In [293]:
exclude = [line.strip() for line in open('/home/aivan/work/results/2022Jul23/list.astex_diverse').readlines()]

In [294]:
exclude += [line.strip() for line in open('/home/aivan/work/results/2022Jul23/list.astex_nonnat').readlines()]
exclude += [line.strip() for line in open('/home/aivan/work/results/2022Jul23/list.casf2016').readlines()]

In [295]:
len(exclude)

436

In [296]:
exclude = list(set(exclude))

In [297]:
len(exclude)

366

In [298]:
df2 = pd.read_csv('/projects/ml/TrRosetta/PDB-2021AUG02/list_v02.csv')

In [299]:
df2['PDBID'] = df2['CHAINID'].apply(lambda x: x.split('_')[0])

In [300]:
found = np.in1d(np.array(exclude),df2['PDBID'].drop_duplicates().values)

In [301]:
sum(found)

366

In [302]:
df_excl = df2[df2['PDBID'].isin(exclude)]

In [303]:
clus_excl = df_excl['CLUSTER'].drop_duplicates().values

In [304]:
df.shape

(360930, 16)

In [305]:
df_filt = df[~df['CLUSTER'].isin(clus_excl)]

In [306]:
df_filt.shape

(310550, 16)

### Exclude GPCRs

Remove clusters containing GPCRs from the GPCR Dock competitions 2008, 2010, 2013, and 2021

In [307]:
import numpy as np

In [308]:
df2 = pd.read_csv('/projects/ml/TrRosetta/PDB-2021AUG02/list_v02.csv')

In [309]:
gpcrs = ['3eml', '3pbl', '3oeu', '3oe0', '3oe6', '3oe8', '3oe9', '4ib4', '4iar', '4iaq', '4jkv', '4n4w', '4qim', '4qin',
         '7vug', '7vuh', '7vgx']

In [310]:
len(gpcrs)

17

In [311]:
df2['PDBID'] = df2['CHAINID'].apply(lambda x: x.split('_')[0])

In [312]:
found = np.in1d(np.array(gpcrs),df2['PDBID'].drop_duplicates().values)

In [313]:
sum(found)

14

In [314]:
df_excl = df2[df2['PDBID'].isin(gpcrs)]

In [315]:
clus_excl = df_excl['CLUSTER'].drop_duplicates().values

In [316]:
df_filt.shape

(310550, 16)

In [317]:
df_filt = df_filt[~df_filt['CLUSTER'].isin(clus_excl)]

In [318]:
df_filt.shape

(308055, 16)

In [319]:
df_filt.to_csv('sm_compl_all_20230211.csv',index=None)

## Mismatched MSAs
MSAs and templates were created a long time ago, and a small # of examples include both a modified residue and its canonical equivalent right next to it, which means they are 1 residue longer than they should be. This is hard to fix without regenerating the MSAs, so we're just going to drop examples with this problem.

### Pilot code

In [ ]:
from data_loader import *
from argparse import Namespace

In [ ]:
import time

In [ ]:
df = pd.read_csv('sm_compl_all_20230211.csv', na_filter=False)

In [ ]:
for col in ['LIGAND','LIGXF','COVALENT','PARTNERS']:
    if col in df:
        df[col] = df[col].apply(lambda x: eval(x))

In [ ]:
df['PDBID'] = df['CHAINID'].apply(lambda x: x.split('_')[0])

In [ ]:
df.shape

(283205, 15)

In [ ]:
df2 = pd.read_csv('/projects/ml/TrRosetta/PDB-2021AUG02/list_v00.csv')
df2['HASH'] = df2['HASH'].apply(lambda x: f'{x:06d}')

In [ ]:
chid2hash = dict(zip(df2['CHAINID'],df2['HASH']))

In [ ]:
records = []
for i,row in df.iterrows():
    for p in row['PARTNERS']:
        if p[-1] != 'polypeptide(L)': continue
        
        newrow = row[['CHAINID','LIGAND','ASSEMBLY']].copy()
        newrow['CHAINID2'] = row['PDBID']+'_'+p[0]
        if newrow['CHAINID2'] in chid2hash:
            newrow['HASH2'] = chid2hash[newrow['CHAINID2']]
        records.append(newrow)

In [ ]:
df3 = pd.DataFrame.from_records(records)

In [ ]:
df3.shape

(703663, 5)

In [ ]:
df3.to_csv('sm_compl_all_chainid_expanded_20230220.csv')

In [ ]:
df = pd.read_csv('sm_compl_all_chainid_expanded_20230220.csv')

In [ ]:
df['PDBID'] = df['CHAINID'].apply(lambda x: x.split('_')[0])

In [ ]:
df['HASH2'] = df['HASH2'].apply(lambda x: f'{int(x):06d}' if pd.notnull(x) else np.nan)

In [ ]:
df.shape

(703663, 7)

In [ ]:
df = df.drop_duplicates(['PDBID','CHAINID2','HASH2'])

In [ ]:
df.shape

(162123, 7)

In [ ]:
from data_loader import *

In [ ]:
params = set_data_loader_params(Namespace())

In [ ]:
t0 = time.time()
ct = 0

pdbid_prev = None

records = []
for i,item in df.iloc[29200:29400].iterrows():

    pdb_chain, pdb_hash = item['CHAINID2'], item['HASH2']
    pdb_id, i_ch_prot = pdb_chain.split('_')

    if pdb_id!=pdbid_prev:
        chains,asmb,covale,modres = pickle.load(gzip.open(f'/projects/ml/RF2_allatom/rcsb/pkl/{pdb_id[1:3]}/{pdb_id}.pkl.gz'))
        ligands, lig_covale = get_ligands(chains, covale)

    # transform doesn't actually matter but we need it for featurizing coords
    i_a = str(item['ASSEMBLY'])
    asmb_xfs = asmb[i_a]
    for ch_xf in asmb_xfs:
        if ch_xf[0] == i_ch_prot:
            break

    # load coords
    ch = chains[i_ch_prot]
    xyz_prot, mask_prot, seq_prot, chid_prot, resi_prot, _ = cif_prot_to_xyz(ch, ch_xf, modres)
    protein_L, nprotatoms, _ = xyz_prot.shape

    # load msa
    if type(pdb_hash) is not str and np.isnan(pdb_hash):
        item['PROT_LEN'] = 0
    else:
        a3mA = get_msa(params['PDB_DIR'] + '/a3m/'+pdb_hash[:3] + '/'+ pdb_hash + '.a3m.gz', pdb_hash)
        item['PROT_LEN'] = xyz_prot.shape[0]
        item['MATCHED'] = a3mA['msa'].shape[1] == item['PROT_LEN']
    records.append(item)

    pdbid_prev = pdb_id
    
    ct += 1

    if ct % 50 == 0:
        print(ct, time.time() - t0)

50 56.29222059249878
100 101.51658630371094
150 119.64641118049622
200 175.9763958454132


In [ ]:
df2 = pd.DataFrame.from_records(records)

In [ ]:
df2.to_csv('bad_msas{args.istart}

### Run on digs

In [ ]:
num = 200

In [ ]:
with open('cmds_filter_bad_msas.txt','w') as outf:
    for istart in range(0, len(df), num):
        if not os.path.exists(f'results_filter_msas/bad_msas{istart}.csv'):
            print(f'source activate SE3-nvidia; python filter_bad_msas.py -istart {istart} -num {num} ',
                  file=outf)

In [ ]:
for istart in range(0, len(df), num):
    if not os.path.exists(f'results_filter_msas/bad_msas{istart}.csv'):
        print(istart)

In [ ]:
df2 = pd.concat([pd.read_csv(f'results_filter_msas/bad_msas{istart}.csv')
                 for istart in range(0, len(df), num)])

In [ ]:
df2.shape

(162123, 10)

In [ ]:
df2['MATCHED'].sum()

162120

In [ ]:
df2[df2['PROT_LEN']==0].shape

(0, 10)

In [ ]:
((df2['PROT_LEN']>0) & (df2['MATCHED']==False)).sum()

3

In [ ]:
df2.to_csv('bad_msas.csv',index=None)

### Filter datasets

In [ ]:
df = pd.read_csv('sm_compl_all_chainid_expanded_20230220.csv')

In [ ]:
df.shape

(703663, 6)

In [ ]:
df['PDBID'] = df['CHAINID'].apply(lambda x: x.split('_')[0])

In [ ]:
df2 = pd.read_csv('bad_msas.csv')

In [ ]:
df2.shape

(162123, 10)

In [ ]:
df2 = df2[df2['MATCHED']==True]

In [ ]:
df2.shape

(162120, 10)

In [ ]:
matched = set(df2['CHAINID2'].values)

In [ ]:
len(matched)

162120

In [ ]:
df = pd.read_csv('sm_compl_all_20230211.csv')

In [ ]:
df.shape

(283205, 15)

In [ ]:
df.drop_duplicates(['CHAINID','LIGAND','ASSEMBLY']).shape

(283205, 15)

In [ ]:
df3 = df[df['CHAINID'].isin(matched)]

In [ ]:
df3.shape

(283202, 15)

In [ ]:
df3['PARTNERS'] = df3['PARTNERS'].apply(lambda x: eval(x))

/scratch/jue/29531735/ipykernel_896644/2734829755.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['PARTNERS'] = df3['PARTNERS'].apply(lambda x: eval(x))


In [ ]:
df3['NUMPARTNERS'] = df3['PARTNERS'].apply(lambda x: len(x))

/scratch/jue/29531735/ipykernel_896644/447582296.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['NUMPARTNERS'] = df3['PARTNERS'].apply(lambda x: len(x))


In [ ]:
df3['PARTNERS'] = df3.apply(
    lambda row: [p for p in row['PARTNERS'] \
                      if p[-1]!='polypeptide(L)' or \
                      (p[-1]=='polypeptide(L)' and (row['CHAINID'].split('_')[0]+'_'+p[0] in matched))],
    axis=1
)

/scratch/jue/29531735/ipykernel_896644/2549799873.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['PARTNERS'] = df3.apply(


In [ ]:
df4 = df3[df3.apply(lambda row: len(row['PARTNERS'])!=row['NUMPARTNERS'], axis=1)]

In [ ]:
df4.shape

(0, 16)

In [ ]:
df3 = df3.drop('NUMPARTNERS',axis=1)

In [ ]:
df3.to_csv('sm_compl_all_goodmsa_20230211.csv',index=None)

## Filter nonbio, nucleic acids

In [118]:
df = pd.read_csv('sm_compl_all_goodmsa_20230211.csv')

In [120]:
df.shape

(283202, 15)

In [121]:
df['LIGAND'] = df['LIGAND'].apply(lambda x: eval(x))
df['COVALENT'] = df['COVALENT'].apply(lambda x: eval(x))

In [122]:
biolip_list = ['NUC','ZN','CA','MG','III','MN','FE','CU','SF4','FE2','CO','FES','GOL','NA',  
            'CL','K','CU1','GOL','XE','NO2','EDO','NI','BR','CD','O','CS','NO','TL','HG','UNL','KR',
            'SR','RB','F','AG','AR','U','AU','MO','SE','GD','YB','VX','SM','LI','RE','N','W','OS','HO','PI']
LA_list = ['EDO','PG4','OGA','SO4','HEZ','FEO','CL','DMS','ACT','MPD','GOL' ,'NH2','CUA','SIW','PGW','IOD','BR','3NI','ZRW','78M','UNX','nan']
rohith = ['MES','CCN','PO4']
metals = ['LA','NI','3CO','K','CR','ZN','CD','PD','TB','YT3','OS','EU','NA','RB','W','YB','HO3',
          'CE','MN','TL','LI','MN3','AU3','AU','EU3','AL','3NI','FE2','PT','FE','CA','AG','CU1',
          'LU','HG','CO','SR','MG','PB','CS','GA','BA','SM','SB','CU','MO','CU2']

In [123]:
exclude = set(biolip_list+LA_list+rohith)-set(metals)

In [124]:
df['LIGANDNAME'] = df['LIGAND'].apply(lambda x: x[0][2])

Doublecheck that non-biological query ligands are filtered.

In [125]:
df = df[~df['LIGANDNAME'].isin(exclude)]

In [126]:
df.shape

(283202, 16)

In [127]:
df = df.drop('LIGANDNAME',axis=1)

Also remove partner ligands on the exclusion list

In [128]:
df['PARTNERS'] = df['PARTNERS'].apply(lambda x: eval(x))

In [129]:
def func(partners):
    return [
        p for p in partners 
        if (p[-1]!='nonpoly') or \
           ((p[-1]=='nonpoly') and all([lig[2] not in exclude for lig in p[0]]))
    ]       

In [130]:
df['PARTNERS'] = df['PARTNERS'].apply(func)

What about DNA/RNA partners? Are there a lot of these?

In [131]:
df2 = df[df['PARTNERS'].apply(lambda partners: any([(p[-1] != 'nonpoly') and (p[-1] != 'polypeptide(L)')
                                              for p in partners]))]

In [132]:
df2.shape

(0, 15)

Remove examples where DNA/RNA are the main partner

In [133]:
df.shape

(283202, 15)

In [134]:
df_ = df[df['PARTNERS'].apply(lambda partners: (partners[0][-1]=='polypeptide(L)') or (partners[0][-1]=='nonpoly'))]

In [135]:
df_.shape

(283202, 15)

In [136]:
df = df_

In [137]:
df.to_csv('sm_compl_all_goodmsa_filt_20230211.csv',index=None)

## Truncate partner lists
Some partner lists are very long (hundreds of interacting chains/ligands). This will definitely not fit into the eventual crop and wastes compute. Let's truncate partner lists so they contain only the most interacting / closest ligands & protein chains

In [138]:
df = pd.read_csv('sm_compl_all_goodmsa_filt_20230211.csv')

In [139]:
for col in ['LIGAND','LIGXF','COVALENT','PARTNERS']:
    if col in df:
        df[col] = df[col].apply(lambda x: eval(x))

In [140]:
df['NUMLIGPARTNERS'] = df['PARTNERS'].apply(lambda x: len([p for p in x if p[-1]=='nonpoly']))

In [141]:
df['NUMPROTPARTNERS'] = df['PARTNERS'].apply(lambda x: len([p for p in x if p[-1]=='polypeptide(L)']))

In [142]:
df.shape

(283202, 17)

In [143]:
df[df['NUMLIGPARTNERS']<20].shape

(271543, 17)

In [144]:
df[df['NUMPROTPARTNERS']<10].shape

(281098, 17)

After some manual inspection I found a bunch of cases where the partners list is long because it contains many instances of 'DOD', which is deuterated water. These should simply be removed.

In [145]:
df['PARTNERS'] = df['PARTNERS'].apply(lambda x: [p for p in x if not (p[-1]=='nonpoly' and p[0][0][2]=='DOD')])
df['NUMLIGPARTNERS'] = df['PARTNERS'].apply(lambda x: len([p for p in x if p[-1]=='nonpoly']))
df['NUMPROTPARTNERS'] = df['PARTNERS'].apply(lambda x: len([p for p in x if p[-1]=='polypeptide(L)']))

Now let's truncate partner lists so they contain a maximum of 20 ligands and 10 proteins

In [146]:
df[df['NUMLIGPARTNERS']>20].shape

(0, 17)

In [147]:
df[df['NUMPROTPARTNERS']>10].shape

(0, 17)

In [148]:
def func(partners):
    newpartners = []
    prot_ct = 0
    prot_num = 10
    lig_ct = 0
    lig_num = 20
    for p in partners:
        if p[-1]=='polypeptide(L)' and prot_ct < prot_num: 
            newpartners.append(p)
            prot_ct+=1
        elif p[-1]=='nonpoly' and lig_ct < lig_num: 
            newpartners.append(p)
            lig_ct+=1
    return newpartners

In [149]:
df['PARTNERS'] = df['PARTNERS'].apply(func)

In [150]:
df['NUMLIGPARTNERS'] = df['PARTNERS'].apply(lambda x: len([p for p in x if p[-1]=='nonpoly']))
df['NUMPROTPARTNERS'] = df['PARTNERS'].apply(lambda x: len([p for p in x if p[-1]=='polypeptide(L)']))

In [151]:
df[df['NUMLIGPARTNERS']>20].shape

(0, 17)

In [152]:
df[df['NUMPROTPARTNERS']>10].shape

(0, 17)

In [155]:
df = df.drop(['NUMLIGPARTNERS','NUMPROTPARTNERS'],axis=1)

### Remove ligands missing some transforms

Initial attempts to run the block below showed that a small number of ligands have more residues in their `LIGAND` entry than transforms in `LIGXF`, because some of the residues have 0 atom occupancy. On manual inspection these examples are missing atoms even from the ligand residues that exist, so let's filter them out completely.

In [157]:
def func(row):
    ligand = row['LIGAND']
    xf_chs = [xf[0] for xf in row['LIGXF']]
    return any([(res[0] not in xf_chs) for res in ligand])

In [158]:
df = df[~df.apply(func, axis=1)]

In [159]:
df.shape

(283202, 15)

There are also some partner ligands that don't have transforms for all the ligand residues.
There are only a few cases, and they all look bad upon manual inspection so we are removing them

In [160]:
npart = df['PARTNERS'].apply(lambda x: len(x))

In [161]:
def func(partners): 
    # remove partners that don't have transforms for all ligand residues
    newpartners = []
    for p in partners:
        if p[-1]=='nonpoly': 
            chxfs = [xf[0] for xf in p[1]]
            if any([res[0] not in chxfs for res in p[0]]):
                continue
        newpartners.append(p)
    return newpartners

In [162]:
df['PARTNERS'] = df['PARTNERS'].apply(func)

In [163]:
npart2 = df['PARTNERS'].apply(lambda x: len(x))

In [164]:
df[npart!=npart2]

,CHAINID,DEPOSITION,RESOLUTION,HASH,CLUSTER,SEQUENCE,LEN_EXIST,LIGAND,ASSEMBLY,COVALENT,LIGXF,PARTNERS,LIGATOMS,LIGATOMS_RESOLVED,SUBSET


In [165]:
df.to_csv('sm_compl_all_goodmsa_partnersfilt_20230211.csv',index=None)

## Filter on contacts, remove nucleic acid examples

In [29]:
df = pd.read_csv('sm_compl_all_goodmsa_partnersfilt_20230211.csv')

In [472]:
for col in ['LIGAND','LIGXF','COVALENT','PARTNERS']:
    if col in df:
        df[col] = df[col].apply(lambda x: eval(x))

Remove ligands that don't make many contacts with proteins. Tweaked the filter below a few times after manual inspection to arrive at a conservative criterion for ligands "floating in space"

In [473]:
# # this kept all metals, even if they made few contacts
# # decided to go with the more stringent filter below after some manual inspection
# idx = df.apply(lambda row: 
#                    (row['LIGAND'][0][2] in metals) or \
#                    (sum([p[2] for p in row['PARTNERS']])>10) or \
#                    len(row['COVALENT'])>0,
#                    axis=1)
# # see what would be filtered out
# df2 = df[~idx]

In [474]:
df2.shape

(11851, 15)

In [166]:
idx = df.apply(lambda row: 
                   (sum([p[2] for p in row['PARTNERS']])>10) or \
                   len(row['COVALENT'])>0,
                   axis=1)

In [167]:
# see what would be filtered out
df2 = df[~idx]

In [168]:
df2.shape

(0, 15)

In [478]:
# apply the filter
df = df[idx]

In [169]:
df.shape

(283202, 15)

Remove examples with DNA/RNA partners. Originally I planned to just ignore these partners but they make up pretty large portions of certain assemblies. We should wait until we can featurize these properly, and then add them back in.

Since I already removed nucleic acid partners, I need to use a previous list to identify these examples, and then remove them from the current list

In [170]:
df3 = pd.read_csv('sm_compl_all_goodmsa_20230211.csv')

In [171]:
df3.shape

(283202, 15)

In [172]:
df3['PARTNERS'] = df3['PARTNERS'].apply(lambda x: eval(x))

In [173]:
idx = df3['PARTNERS'].apply(
    lambda partners: any([p[-1]!='nonpoly' and p[-1]!='polypeptide(L)' for p in partners]))

In [174]:
idx.sum()

0

In [175]:
df['LIGAND'] = df['LIGAND'].astype(str)

In [176]:
df.drop_duplicates(['CHAINID','LIGAND','ASSEMBLY']).shape

(283202, 15)

In [177]:
to_remove = df3[idx]

In [178]:
to_remove['LIGAND'] = to_remove['LIGAND'].astype(str)

In [179]:
to_remove['to_remove']=True

In [180]:
df_ = df.merge(to_remove[['CHAINID','LIGAND','ASSEMBLY','to_remove']], on=['CHAINID','LIGAND','ASSEMBLY'], how='left')

In [181]:
df_.shape

(283202, 16)

In [182]:
(df_['to_remove']==True).sum()

0

In [183]:
df = df_[df_['to_remove']!=True]

In [184]:
df.shape

(283202, 16)

In [185]:
df.to_csv('sm_compl_all_finalfilt_20230211.csv',index=None)

### Remove partners that aren't query ligands
I've filtered out some non-contacting ligands above. Remove these from partner lists as well, so we're never training on anything that fails our filters for query ligands.

NOTE: Chose not to do this for now, as this would remove many ligands that seem good.

In [55]:
df = pd.read_csv('sm_compl_all_finalfilt_20230211.csv')

In [6]:
df['PARTNERS'] = df['PARTNERS'].apply(lambda x: eval(x))

In [7]:
df.shape

(283205, 16)

In [8]:
pdbids = df['PDBID'].drop_duplicates().values

In [9]:
len(pdbids)

64239

This takes about 20 minutes

In [10]:
ct = 0
num = 1000
t0 = time.time()
records = []
for pdbid in pdbids:

    tmp = df[df['PDBID']==pdbid]
    ligands = set(tmp['LIGAND'].values)

    for i,row in tmp.iterrows():
        row['NEWPARTNERS'] = [p for p in row['PARTNERS'] if p[-1]=='polypeptide(L)' or str(p[0]) in ligands]
        records.append(row)
        
    if ct % num == 0:
        print(ct, time.time()-t0)
    ct += 1

0 0.08199191093444824
1000 14.8564133644104
2000 29.866195917129517
3000 44.136276960372925
4000 58.463584899902344
5000 72.63599705696106
6000 86.80351328849792
7000 101.36547660827637
8000 115.69562816619873
9000 129.8228840827942
10000 144.0588080883026
11000 158.40963530540466
12000 172.46609711647034
13000 186.56121516227722
14000 200.58768224716187
15000 215.1282877922058
16000 230.29100489616394
17000 244.8010184764862
18000 259.4128499031067
19000 273.6325685977936
20000 287.8356671333313
21000 302.297251701355
22000 316.42806029319763
23000 330.7955467700958
24000 345.0605618953705
25000 359.36014008522034
26000 373.5586075782776
27000 388.1970980167389
28000 402.48542523384094
29000 416.83008551597595
30000 431.002414226532
31000 445.42239356040955
32000 460.90906858444214
33000 475.42472434043884
34000 489.91762113571167
35000 503.9820399284363
36000 518.3493394851685
37000 532.6180474758148
38000 546.7932860851288
39000 561.428995847702
40000 575.6421086788177
41000 589.785

In [11]:
df2 = pd.DataFrame.from_records(records)

In [12]:
idx = df2['PARTNERS'].apply(lambda x: len(x)) != df2['NEWPARTNERS'].apply(lambda x: len(x))

In [13]:
df2[idx].shape

(90856, 17)

In [16]:
df3 = df2[idx]

In [14]:
df.shape

(283205, 16)

Checked some examples. Certain things are getting removed by PDBBind/BioLip filters that seem like good ligands, such as Na ions.

Let's not use this filter for now.

In [234]:
# df.to_csv('sm_compl_all_finalfilt_20230127.csv',index=None)

## Label data subsets

In [186]:
df = pd.read_csv('sm_compl_all_finalfilt_20230211.csv')

In [187]:
for col in ['LIGAND','LIGXF','COVALENT','PARTNERS']:
    if col in df:
        df[col] = df[col].apply(lambda x: eval(x))

In [188]:
df.shape

(283202, 16)

For some reason there are entries with duplicated covalent bonds. deduplicate these.

In [189]:
tmp = df[df['COVALENT'].apply(lambda x: len(x)!=len(set(x)))]

In [190]:
tmp.shape

(0, 16)

In [191]:
def func(covalent):
    new_covalent = []
    for bond in covalent:
        if bond not in new_covalent:
            new_covalent.append(bond)
    return new_covalent

In [192]:
df['COVALENT'] = df['COVALENT'].apply(func)

In [193]:
tmp = df[df['COVALENT'].apply(lambda x: len(x)!=len(set(x)))]

In [194]:
tmp.shape

(0, 16)

In [195]:
df = df.drop(['PDBID','to_remove'],axis=1)

KeyError: "['PDBID'] not found in axis"

In [197]:
df.shape

(283202, 15)

### Assemblies
After some exploratory analysis I chose the following definition of assembly. These examples will be in their own dataset that will always attempt to featurize every single ligand in the partner list.

The remaining examples will be put in datasets that will only attempt to featurize the query ligand and its main interacting protein chain.

In [198]:
idx = df['PARTNERS'].apply(
    lambda partners: 
        len([p for p in partners if p[-1]=='nonpoly' and p[2]>0]) + \
        len([p for p in partners if p[-1]=='polypeptide(L)' and p[2]>10]) > 1)

In [199]:
idx.sum()

151807

In [200]:
df.loc[idx,'SUBSET'] = 'asmb'

### Multi-residue

In [201]:
idx = df['LIGAND'].apply(lambda x: len(x)>1) & df['COVALENT'].apply(lambda x: len(x)==0) & df['SUBSET'].isnull()

In [202]:
idx.sum()

0

In [203]:
df.loc[idx,'SUBSET'] = 'multi'

### Covalent

In [204]:
idx = df['COVALENT'].apply(lambda x: len(x)>0) & df['SUBSET'].isnull()

In [205]:
idx.sum()

0

In [206]:
df.loc[idx,'SUBSET'] = 'covale'

### Metal ions

In [207]:
df['LIGANDNAME'] = df['LIGAND'].apply(lambda x: x[0][2])

In [208]:
metals = ['LA','NI','3CO','K','CR','ZN','CD','PD','TB','YT3','OS','EU','NA','RB','W','YB','HO3',
          'CE','MN','TL','LI','MN3','AU3','AU','EU3','AL','3NI','FE2','PT','FE','CA','AG','CU1',
          'LU','HG','CO','SR','MG','PB','CS','GA','BA','SM','SB','CU','MO','CU2']

In [209]:
idx = df['LIGANDNAME'].isin(metals) & df['SUBSET'].isnull()

In [210]:
idx.sum()

0

In [211]:
df.loc[idx,'SUBSET'] = 'metal'

In [212]:
df = df.drop(['LIGANDNAME'],axis=1)

### Organic ligands
What is left at this point is just the "basic" ligand-protein complexes

In [213]:
idx = df['SUBSET'].isnull()

In [214]:
idx.sum()

0

In [215]:
df.loc[idx,'SUBSET'] = 'organic'

In [216]:
df.to_csv('sm_compl_all_202302120.csv',index=None)

### Various bad examples

These cause errors during data-loading or forward pass that are due to various idiosyncratic data problems with the original PDB files. To fix them would require manual tweaking so we will exclude them for convenience.

In [220]:
df = pd.read_csv('sm_compl_all_202302120.csv')

In [221]:
df.shape

(283202, 15)

In [222]:
df[df['CHAINID'].str.contains('6ofk')]

,CHAINID,DEPOSITION,RESOLUTION,HASH,CLUSTER,SEQUENCE,LEN_EXIST,LIGAND,ASSEMBLY,COVALENT,LIGXF,PARTNERS,LIGATOMS,LIGATOMS_RESOLVED,SUBSET
14548,6ofk_A,2019-03-30,1.15,19511,22071,GHHHHHHSSGGKLPVPWPTLVTTLXVQCFSRYPDHMKRHDFFKSAM...,225,"[('C', '301', 'CRO')]",1,"[(('A', '24', 'LEU', 'C'), ('C', '301', 'CRO',...","[('C', 1)]","[('A', 0, 339, 1.3623689413070679, 'polypeptid...",22,22,covale
14549,6ofk_B,2019-03-30,1.15,19511,22071,GHHHHHHSSGGKLPVPWPTLVTTLXVQCFSRYPDHMKRHDFFKSAM...,228,"[('D', '301', 'CRO')]",2,"[(('B', '24', 'LEU', 'C'), ('D', '301', 'CRO',...","[('D', 1)]","[('B', 0, 340, 1.3511673212051392, 'polypeptid...",22,22,covale


In [224]:
df = df[~df['CHAINID'].isin([
    '1q9x_K', '4s0n_A', '3agv_A', '5l6x_B', '5l6x_A',
    '1khz_B', '1g9q_A', '1g9q_B', # cuda indexing errors during forward pass
    '4u9i_B', '4u9h_B', '4jhq_A', '4jhq_B', '5myq_A', '5myq_B', # error during loading
    '5myq_C', '5myq_D', '6g7r_D', '6g7r_B', '6gal_D', '6fpi_B', # error during loading
    '6fpi_D', '6fpw_B', '6fpw_D', # error during loading
    '1adl_A', '1bs3_A', '1bs3_B', '1btx_A', '1bxw_A', '1etu_A', '1gjm_A',
    '1h3v_B', '1jkj_B', '1l0i_A', '1q1k_A', '1qga_A', '1qga_B', '1nte_A',
    '1x83_B', '2b4b_B', '3dpm_A', '3dpm_B', '4ztt_F', '5kxd_A', '6mhb_F',
    '4mm1_B','5m29_B','6tzk_A',
    '6ofk_A','6ofk_B', # GFP chromophore got curated incorrectly. this can't be featurized easily with our current code so skip, but revisit later
])]

In [225]:
df.shape

(283126, 15)

In [226]:
df.to_csv('sm_compl_all_20230220.csv',index=None)

## Strict validation set
2023-1-16

Re-create the strict validation set with no tanimoto overlap with train

### Functions

In [111]:
import pickle, gzip, time
from openbabel import openbabel
sys.path.insert(0,'/home/jue/git/rf2a-fd3/')
import chemical
from chemical import atom_num, frame_priority2atom

In [97]:
def cif_ligand_to_xyz(atoms, asmb_xfs=None, ch2xf=None):

    elnum_to_atom = dict(zip(atom_num, frame_priority2atom))
    atoms_no_H = {k:v for k,v in atoms.items() if v.element != 1} # exclude hydrogens
    L = len(atoms_no_H)

    xyz = torch.zeros(L, 3)
    mask = torch.zeros(L,).bool()
    seq = torch.full((L,), np.nan)
    chid = ['-']*L
    akeys = [None]*L
    
    # create coords, atom mask, and seq tokens
    for i,(k,v) in enumerate(atoms_no_H.items()):
        xyz[i, :] = torch.tensor(v.xyz)
        mask[i] = v.occ
        if v.element not in elnum_to_atom:
            print('Element not in alphabet:',v.element)
            seq[i] = chemical.aa2num['ATM']
        else:
            seq[i] = chemical.aa2num[elnum_to_atom[v.element]]
        akeys[i] = k
        chid[i] = k[0]
        
    if asmb_xfs is not None:
        # apply transforms
        chid = np.array(chid)
        for i_ch in np.unique(chid):
            idx = chid==i_ch
            xf = torch.tensor(asmb_xfs[ch2xf[i_ch]][1]).float()
            u,r = xf[:3,:3], xf[:3,3]
            xyz[idx] = torch.einsum('ij,aj->ai', u, xyz[idx]) + r[None,None]
        
    return xyz, mask, seq, chid, akeys

def cif_ligand_to_obmol(xyz, akeys, atoms, bonds):
    
    mol = openbabel.OBMol()    
    for i,k in enumerate(akeys):
        a = mol.NewAtom()
        a.SetAtomicNum(atoms[k].element)
        a.SetVector(float(xyz[i,0]), float(xyz[i,1]), float(xyz[i,2]))

    sm_L = len(akeys)
    bond_feats = torch.zeros((sm_L,sm_L))
    for bond in bonds:
        if bond.a not in akeys or bond.b not in akeys: continue # intended to skip bonds to H's
        i = akeys.index(bond.a)
        j = akeys.index(bond.b)
        bond_feats[i,j] = bond.order if not bond.aromatic else 4
        bond_feats[j,i] = bond_feats[i,j]

        obb = openbabel.OBBond()
        obb.SetBegin(mol.GetAtom(i+1))
        obb.SetEnd(mol.GetAtom(j+1))
        obb.SetBondOrder(bond.order)
        if bond.aromatic:
            obb.SetAromatic()

        mol.AddBond(obb)
    
    return mol, bond_feats

### Get ligand names

In [279]:
df = pd.read_csv('sm_compl_all_goodmsa_20230116.csv')

for col in ['LIGAND','PARTNERS','LIGXF']:
    df[col] = df[col].apply(lambda x: eval(x))

df['LIGNAME'] = df['LIGAND'].apply(lambda x: x[0][2])

In [280]:
df2 = df.drop_duplicates('LIGNAME')

In [281]:
df2.shape

(17468, 14)

In [285]:
df2 = df2[~df2['LIGNAME'].isin(metals)]

In [286]:
df2.shape

(17424, 14)

Also need to compare to ligands in old dataset, since we've been training fold & dock 3 on it

In [314]:
df_old = pd.read_csv('/projects/ml/RF2_allatom/list_v02_sm_filt_notest.csv',index_col=None,dtype=str)

In [315]:
df_old['LIGANDS'] = df_old['LIGANDS'].apply(lambda x: eval(x))

In [316]:
records = []
for i,row in df_old.iterrows():
    for lig in row['LIGANDS']:
        newrow = row.copy()
        newrow['LIGNAME'] = lig.split('_')[1]
        records.append(newrow)

In [317]:
df3 = pd.DataFrame.from_records(records)

In [318]:
df3.shape

(74996, 10)

In [319]:
df3 = df3.drop_duplicates('LIGNAME')

In [320]:
df3.shape

(18229, 10)

In [321]:
df3.to_csv('old_ligands.csv',index=False)

In [322]:
new = set(df3['LIGNAME'].values)-set(df2['LIGNAME'].values)

In [323]:
len(new)

3438

In [324]:
lignames = set(df3['LIGNAME'].values).union(set(df2['LIGNAME'].values))

In [327]:
len(lignames)

20862

In [328]:
df3 = pd.read_csv('ligands.csv')

In [329]:
df3['LIGAND'] = df3['LIGAND'].apply(lambda x: eval(x))

In [330]:
df3['LIGNAME'] = df3['LIGAND'].apply(lambda x: list(x)[0][2])

In [331]:
df3 = df3[df3['LIGNAME'].isin(lignames)].drop_duplicates('LIGNAME')

In [332]:
df3.shape

(20761, 4)

In [334]:
df3.to_csv('ligands_for_tanimoto.csv')

### Copy mol2's of ligands only in old set

About 100 are missing. Are these in the old set?

In [308]:
missing = set(lignames)-set(df3['LIGNAME'])

In [333]:
len(missing)

101

In [335]:
df = pd.read_csv('old_ligands.csv')

In [337]:
df = df[df['LIGNAME'].isin(missing)]

In [338]:
df.shape

(101, 10)

In [343]:
df['LIGANDS'] = df['LIGANDS'].apply(lambda x: eval(x))

Yes, so we'll have to copy over their mol2 files separately.

In [340]:
import shutil

In [364]:
for i,row in df.iterrows():
    for lig in row['LIGANDS']:
        if row['LIGNAME'] in lig:
            break
            
    filenames = glob.glob('/projects/ml/RF2_allatom/by-pdb/'+
        row['CHAINID'][1:3]+'/'+row['CHAINID'][:4]+'_'+row['LIGNAME']+'*.mol2')

    outfn = row['LIGNAME']+'.mol2'

    outdir = 'mol2/'+outfn[0]+'/'
    os.makedirs(outdir, exist_ok=True)

    shutil.copyfile(filenames[0], outdir+outfn)

### Save ligands from cif to mol2

In [365]:
df = pd.read_csv('ligands_for_tanimoto.csv')

In [366]:
df.shape

(20761, 5)

In [370]:
df['LIGAND'] = df['LIGAND'].apply(lambda x: eval(x))

In [371]:
ct = 0
t0 = time.time()
for i,row in df.iterrows():

    pdb_id = row['PDBID']
    ligand = list(row['LIGAND'])

    chains, asmb, covale, modres = pickle.load(gzip.open(f'/projects/ml/RF2_allatom/rcsb/pkl/{pdb_id[1:3]}/{pdb_id}.pkl.gz'))

    lig_atoms = dict()
    lig_bonds = []
    for i_ch,ch in chains.items():
        for k,v in ch.atoms.items():
            if k[:3] in ligand:
                lig_atoms[k] = v
        for bond in ch.bonds:
            if bond.a[:3] in ligand or bond.b[:3] in ligand:
                lig_bonds.append(bond)
    # for bond in covale:
    #     if bond.a[:3] in ligand and bond.b[:3] in ligand:
    #         lig_bonds.append(bond)

    xyz_sm, mask_sm, msa_sm, chid_sm, akeys = cif_ligand_to_xyz(lig_atoms)

    mol, bond_feats_sm = cif_ligand_to_obmol(xyz_sm, akeys, lig_atoms, lig_bonds)

    obConversion = openbabel.OBConversion()
    obConversion.SetOutFormat("mol2")
    outfn = ligand[0][2]+'.mol2'
    outdir = f'mol2/{outfn[0]}/'
    os.makedirs(outdir, exist_ok=True)
    obConversion.WriteFile(mol, outdir+outfn)
    
    ct += 1
    
    if ct % 50 == 0:
        print(ct, time.time() - t0, ligand)

KeyboardInterrupt: 

Put the above in a standalone script to run in parallel on digs.

Generate a list of commands to run

In [372]:
df.shape

(20761, 5)

In [373]:
num = 100

In [374]:
with open('cmds_lig_to_mol2.txt','w') as outf:
    for istart in range(0, len(df), num):
        print(f'source activate dlchem; python lig_to_mol2.py -istart {istart} -num {num}',
              file=outf)

### Compute tanimoto

In [375]:
from openbabel import openbabel, pybel
openbabel.obErrorLog.SetOutputLevel(0) # disable openbabel warnings

In [376]:
filenames = glob.glob('mol2/*/*.mol2')

In [377]:
len(filenames)

20862

In [378]:
t0 = time.time()
mols = [next(pybel.readfile('mol2',fn)) for fn in filenames]
fps = [mol.calcfp() for mol in mols]

In [379]:
sim_s = []
ct = 0
for fp in fps:
    sim = np.array([fp|fp2 for fp2 in fps])
    sim_s.append(sim)
    ct += 1
    if ct%100 == 0:
        print(time.time()-t0)

200.96962189674377
202.82005214691162
204.62919449806213
206.47700548171997
208.45842242240906
210.2435073852539
212.09637117385864
214.01284098625183
215.9991946220398
217.95119738578796
219.68930292129517
221.39617609977722
223.13687133789062
224.90238904953003
226.68455338478088
228.3933811187744
230.19573545455933
232.1125407218933
233.9922971725464
235.85839891433716
237.74592900276184
239.45363926887512
241.20519161224365
242.99975514411926
244.72403478622437
246.4277424812317
248.19553351402283
250.0407178401947
251.89701128005981
253.65857601165771
255.48559498786926
257.20835185050964
258.96954584121704
260.7551484107971
262.45608019828796
264.16799783706665
265.96162700653076
267.79862570762634
269.6092894077301
271.3825283050537
273.1717450618744
274.92721486091614
276.6996970176697
278.4632840156555
280.1786003112793
281.9093804359436
283.71502780914307
285.5757722854614
287.3399143218994
289.14551067352295
290.870304107666
292.6472382545471
294.4373996257782
296.1445569992

In [380]:
sim_s[0].shape

(20862,)

In [381]:
sim = np.stack(sim_s, axis=0)

In [382]:
sim.shape

(20862, 20862)

In [383]:
names = [os.path.basename(fn).replace('.mol2','') for fn in filenames]

In [384]:
np.savez('tanimoto_ligands.npz',names=names,sim=sim)

### Curate strict valid set

In [94]:
val_pdb_ids = set([int(l) for l in open('/projects/ml/RF2_allatom/valid_remapped').readlines()])

In [95]:
# df = pd.read_csv('sm_compl_20230118.csv') # previously used this larger list

In [96]:
# get the "train" split from these sets
df = pd.read_csv('sm_compl_all_20230211.csv'),
for col in ['LIGAND','PARTNERS','LIGXF']:
    df[col] = df[col].apply(lambda x: eval(x))

In [97]:
df_train = df[~df['CLUSTER'].isin(val_pdb_ids)]

In [98]:
df_train.shape

(197771, 16)

In [99]:
def func(row):
    lignames = [lig[2] for lig in row['LIGAND']]
    for p in row['PARTNERS']:
        if p[-1]=='nonpoly':
            lignames += [lig[2] for lig in p[0]]
    return lignames

In [100]:
df_train['LIGNAMES'] = df_train.apply(func, axis=1)

/scratch/jue/28700558/ipykernel_1480857/3330530388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['LIGNAMES'] = df_train.apply(func, axis=1)


In [101]:
train_lignames = set()
for i,row in df_train.iterrows():
    train_lignames.update(row['LIGNAMES'])

In [102]:
len(train_lignames)

16963

In [103]:
# get the "valid" split from these sets
df2 = df[df['SUBSET']=='organic']
df2['LIGNAME'] = df2['LIGAND'].apply(lambda x: x[0][2])

In [104]:
df_valid = df2[df2['CLUSTER'].isin(val_pdb_ids)]

In [105]:
df_valid.shape

(3267, 17)

In [106]:
dat = np.load('tanimoto_ligands.npz')

In [107]:
names = dat['names']

In [108]:
len(names)

20862

In [109]:
sim = dat['sim']

In [110]:
sim.shape

(20862, 20862)

In [111]:
df = pd.read_csv('old_ligands.csv')

In [112]:
df_train_old = df[~df['CLUSTER'].isin(val_pdb_ids)]

In [113]:
train_lignames = train_lignames.union(set(df_train_old['LIGNAME'].values))
name_in_train = np.array([name in train_lignames for name in names])

In [114]:
len(name_in_train)

20862

In [115]:
sum(name_in_train)

19751

In [116]:
name2idx = dict(zip(names,range(len(names))))

In [117]:
def func(ligname):
    return name_in_train[sim[name2idx[ligname]]>0.85].any()

In [118]:
df_valid['TRAIN_OVERLAP'] = df_valid['LIGNAME'].apply(func)

/scratch/jue/28700558/ipykernel_1480857/3353789439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid['TRAIN_OVERLAP'] = df_valid['LIGNAME'].apply(func)


In [119]:
df_strict = df_valid[~df_valid['TRAIN_OVERLAP']]

In [120]:
df_strict.shape

(659, 18)

In [121]:
df_strict['CLUSTER'].drop_duplicates().shape

(61,)

In [122]:
df_strict = df_strict.drop(['LIGNAME','TRAIN_OVERLAP'],axis=1)

In [126]:
df_strict.to_csv('sm_compl_valid_strict_20230211.csv',index=None)